# Kelly Criterion

## Problem 1
A simple gambling game: You start with $100 and repeatedly toss a fair coin and bet against a casino. This casino is super friendly. Everytime you get a heads (H) you win 80% of your bet! When you get tails (T), you lose 50% of your bet.

Arithmetically, this gives us an average earning of $\frac{1}{2}\times0.8+\frac{1}{2}\times-0.5=0.15$. 15% average gain per coin toss! Because this seems like a great deal. Lets always bet all our money.

In [11]:
import numpy as np
import random
import plotly.graph_objects as go
import plotly.express as px

In [12]:
# Simulate the above game 1000 times, each time making 20 bets on the coin
NUM_GAMES = 1000
GAME_LENGTH = 20

games = [[100] for _ in range(NUM_GAMES)]
for game in games:
  for _ in range(GAME_LENGTH):
    win = random.choice([True, False])
    if win:
      game.append(game[-1]*1.8)
    else:
      game.append(game[-1]*0.5)

games = np.array(games)

Plotting the means and medians of the performance of each game:  
  1. We find a huge discrepancy between the mean and median of the performances
  2. The median of actually below 100, most of the games have made a lost!

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter( y = games.mean(axis = 0), name = 'Mean'))
fig.add_trace(go.Scatter( y = np.median(games, axis = 0), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = 100)
fig.show()

Plot the individual games: Here we see the reason for the discrepancy, a small minority of the games (lucky games) end up with huge winnings, skewing the mean.

In [14]:
fig = go.Figure()
for game in games:
  fig.add_trace(go.Scatter( y = game))
  fig.add_trace(go.Scatter( y = games.mean(axis = 0), name = 'Mean', fillcolor= 'red'))
fig.add_trace(go.Scatter( y = np.median(games, axis = 0), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = 100)
fig.show()

Plot the individual games on a log axis

In [15]:
fig = go.Figure()
for game in games:
  fig.add_trace(go.Scatter( y = np.log(game)))
  fig.add_trace(go.Scatter( y = np.log(games.mean(axis = 0)), name = 'Mean', fillcolor= 'red'))
fig.add_trace(go.Scatter( y = np.log(np.median(games, axis = 0)), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = np.log(100))
fig.show()

## Problem 2
Before we saw that betting our entire wealth on each game added up with us leaving on a loss more often than not. This was due to the multiplicative properties of the game -> When we lose we aren't able to make as much even if we win afterwards. instead of betting our entire wealth, lets just bet a fixed amount instead: $50

In [16]:
# Simulate the above game 1000 times, each time making 20 bets on the coin
NUM_GAMES = 1000
GAME_LENGTH = 20

games = [[100] for _ in range(NUM_GAMES)]
for game in games:
  for _ in range(GAME_LENGTH):
    win = random.choice([True, False])
    if win:
      game.append(game[-1] + 50 * 0.8)
    else:
      game.append(game[-1] - 50 * 0.5)

games = np.array(games)

fig = go.Figure()
fig.add_trace(go.Scatter( y = games.mean(axis = 0), name = 'Mean'))
fig.add_trace(go.Scatter( y = np.median(games, axis = 0), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = 100)
fig.show()

Now we can see that the mean and median winnings is equal! Due to our constant bet size, we always have an expected winning of $\frac{1}{2}×0.8×50-\frac{1}{2}×0.5×50=7.5$. We expect to win $7.5 every flip!

In [17]:
fig = go.Figure()
for game in games:
  fig.add_trace(go.Scatter( y = game))
  fig.add_trace(go.Scatter( y = games.mean(axis = 0), name = 'Mean', fillcolor= 'red'))
fig.add_trace(go.Scatter( y = np.median(games, axis = 0), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = 100)
fig.show()

Notice that each step is now additive. Since our bet size is constant (\$50), at each step we only either lose \$25 or gain \$40. This strategy seems great except for a few problems:  
- Strategy breaks down for unlucky few who lose too much in the beginning and end up with negative. If they have less than \$50 they can't keep betting \$50
- This begs the follow up question of what is the optimal bet: Why \$50 and should this vary depending on our total capital?

## Problem 3
Instead of betting a constant value -> we size our bet according to our capital -> bet a fraction of capital (e.g. 10% of capital)

Assuming we bet 10% of our wealth, the expected growth rate (mode/median) per coin flip = expectation of half heads and half tails
$$r = (1+\frac{1}{10}×0.8)^{0.5}×(1-\frac{1}{10}×0.5)^{0.5}=1.013 $$

Notice that the geometric mean of betting 10% of our capital is a positive rate! Now to maximise r, lets generalise the equation.  
Variables:  
- $f = $ bet proportion
- $b = $ gain
- $a = $ loss
- $p = $ probability of heads
- $q = $ probability of tails
$$ r = (1+fb)^p×(1-fa)^q$$ 

In [27]:
def rate_func(f: float, b = 0.8, a = 0.5, p = 0.5, q = 0.5):
  return ((1 + f*b)**p) * ((1 - f*a)**q)

y_vals = []
for f in np.linspace(0,1,100):
  y_vals.append(rate_func(f = f))

fig = px.line(x= np.linspace(0,1,100), y = y_vals, labels = {'x':'f', 'y':'r'}, title='Growth rate as a function of bet proportion')
fig.add_hline(y = 1)
fig.show()

The rate function gives us a nice concave function within the domain of f = [0,1]. Thus we can find the derivative = 0 to find the local maxima.
$$ r = (1+fb)^p×(1-fa)^q$$ 
$$ln(r) = ln((1+fb)^p) + ln((1-fa)^q)$$
$$ln(r) = p ln(1+fb) + q ln(1-fa)$$
$$ \frac{1}{r}\times\frac{dr}{df}=\frac{pb}{1+fb}+\frac{-qa}{1-fa}$$
Set $\frac{dr}{df}=0$
$$0 = r×(\frac{pb}{1+f^*b}+\frac{-qa}{1-f^*a})$$
$$f^* = \frac{p}{a}-\frac{q}{b}$$

Thus the optimal bet size is a function of probabilities of winning/losing as well as the expected returns of each situation. In this game, optimal bet $f^* = \frac{0.5}{0.5}-\frac{0.5}{0.8}=0.375$.   
Running the simulation to visualise our winnings using the optimal bet size

In [28]:
# Simulate the above game 1000 times, each time making 20 bets on the coin
NUM_GAMES = 1000
GAME_LENGTH = 20
BET_SIZE = 0.375

games = [[100] for _ in range(NUM_GAMES)]
for game in games:
  for _ in range(GAME_LENGTH):
    win = random.choice([True, False])
    if win:
      game.append(game[-1] + game[-1] * BET_SIZE * 0.8)
    else:
      game.append(game[-1] - game[-1] * BET_SIZE * 0.5)

games = np.array(games)

fig = go.Figure()
fig.add_trace(go.Scatter( y = games.mean(axis = 0), name = 'Mean'))
fig.add_trace(go.Scatter( y = np.median(games, axis = 0), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = 100)
fig.show()

In [29]:
fig = go.Figure()
for game in games:
  fig.add_trace(go.Scatter( y = game))
  fig.add_trace(go.Scatter( y = games.mean(axis = 0), name = 'Mean', fillcolor= 'red'))
fig.add_trace(go.Scatter( y = np.median(games, axis = 0), name = 'Median'))
fig.update_layout(xaxis_title = 'Bets', yaxis_title = '$')
fig.add_hline(y = 100)
fig.show()